#  SWB211 Project 

## Joint Density Calculation

### Imports

In [1]:
import pandas as pd 
import os

#os.chdir(r"G:\My Drive\0.SWB\SWB211")  # Provide the new path here          # I don't think this is necessary anymore, delete?
#print(os.getcwd()) # Prints the current working directory                   # I don't think this is necessary anymore, delete?
 

### Path Management
using [`os.path.join`](https://docs.python.org/3/library/os.path.html), to fit the paths for Linux/MacOS and Windows syntax, which is more robust:
* Linux/MacOS syntax (e.g. `../folder_name`) 
* and Windows syntax (e.g. `..\\folder_name`) 

In [2]:
csv_path_population_block = os.path.join('.', 'data', 'census_data', 'nhgis0202_csv', 'nhgis0202_ds248_2020_block.csv')
csv_path_education_income = os.path.join('.', 'data', 'census_data', 'nhgis0197_csv', 'nhgis0197_ds254_20215_blck_grp.csv')
csv_path_age = os.path.join('.', 'data', 'census_data', 'nhgis0203_csv', 'nhgis0203_ds254_20215_blck_grp.csv')

dta_path_6_counties = os.path.join('.', 'data', 'intermediate_data', 'b_KC_6counties.dta')
dta_path_county_crosswalk = os.path.join('.', 'data', 'intermediate_data', 'b_bg_county_crosswalk.dta')

### DataFrame of population count per house block

In [3]:
block_raw = pd.read_csv(csv_path_population_block, low_memory=False)
block_raw.columns = block_raw.columns.str.lower()

block_data = block_raw[['gisjoin', 'state','county','u7b001']]
block_data = block_data.rename(columns={'gisjoin': 'b_gisjoin'})                
block_data = block_data.rename(columns={'u7b001': 'b_population'}) # b_target_pop: Block level estimate of total population.

block_data            

,b_gisjoin,state,county,b_population
0,G20000109526001000,Kansas,Allen County,3
1,G20000109526001001,Kansas,Allen County,8
2,G20000109526001002,Kansas,Allen County,7
3,G20000109526001003,Kansas,Allen County,0
4,G20000109526001004,Kansas,Allen County,0
...,...,...,...,...
426156,G29051001278002030,Missouri,St. Louis city,0
426157,G29051001278002031,Missouri,St. Louis city,3
426158,G29051001278002032,Missouri,St. Louis city,0
426159,G29051001278002033,Missouri,St. Louis city,0


### DataFrame of household income and individual's educational status accumulated per block group
A block group is a collection of several house blocks; approximatly 30-50 blocks form a block group.

In [4]:
blockgroup_raw = pd.read_csv(csv_path_education_income)
blockgroup_raw.columns = blockgroup_raw.columns.str.lower()
col_list=blockgroup_raw[['aop8e002', 'aop8e003', 'aop8e004', 'aop8e005', 'aop8e006', 'aop8e007', 'aop8e008' ,'aop8e009', 'aop8e010', 'aop8e011', 'aop8e012', 'aop8e013', 'aop8e014', 'aop8e015', 'aop8e016', 'aop8e017', 'aop8e018', 'aop8e019', 'aop8e020', 'aop8e021']]
blockgroup_raw['bg_educ_lt_bachelars']=col_list.sum(axis=1) # summing population count below bachelor's degree
col_list=blockgroup_raw[['aoqhe002', 'aoqhe003', 'aoqhe004', 'aoqhe005', 'aoqhe006', 'aoqhe007', 'aoqhe008', 'aoqhe009']]
blockgroup_raw['bg_inc_lt_45000']=col_list.sum(axis=1) # summing housdhold inclome below 45 k

blockgroup_raw['s_bg_educ_lt_bachelars']=blockgroup_raw['bg_educ_lt_bachelars']/blockgroup_raw['aop8e001'] # percentage of non-BA's over whole population (creates Nan values where the population per block group is 0)
blockgroup_raw['s_bg_inc_lt_45000']=blockgroup_raw['bg_inc_lt_45000']/blockgroup_raw['aoqhe001'] # percentage of low earning households over all housholds (creates Nan values where the population per block group is 0)
blockgroup_raw = blockgroup_raw.rename(columns={'gisjoin': 'bg_gisjoin'})                
bg_educ_inc=blockgroup_raw[['bg_gisjoin', 'bg_educ_lt_bachelars','bg_inc_lt_45000','s_bg_educ_lt_bachelars','s_bg_inc_lt_45000']]

bg_educ_inc

,bg_gisjoin,bg_educ_lt_bachelars,bg_inc_lt_45000,s_bg_educ_lt_bachelars,s_bg_inc_lt_45000
0,G20000109526001,1146,195,0.861007,0.281792
1,G20000109527001,548,210,0.734584,0.466667
2,G20000109527002,527,110,0.780741,0.303867
3,G20000109528001,521,240,0.785822,0.551724
4,G20000109528002,650,252,0.672878,0.464945
...,...,...,...,...,...
7487,G29051001277001,661,211,0.937589,0.703333
7488,G29051001277002,654,289,1.000000,0.865269
7489,G29051001277003,176,79,0.752137,0.576642
7490,G29051001278001,959,982,0.762928,0.891916


### DataFrame of population's age accumulated per block group
A block group is a collection of several house blocks; approximatly 30-50 blocks form a block group.

In [5]:
blockgroup_raw = pd.read_csv(csv_path_age)
blockgroup_raw.columns = blockgroup_raw.columns.str.lower()
blockgroup_raw.describe()
print(blockgroup_raw.info())
blockgroup_raw.dtypes 
col_list=blockgroup_raw[['aonte007', 'aonte008', 'aonte009', 'aonte010', 'aonte011', 'aonte012', 'aonte013', 'aonte014', 'aonte015', 'aonte016' ,'aonte017', 'aonte018', 'aonte019' ,'aonte020', 'aonte021', 'aonte022', 'aonte023' ,'aonte024' ,'aonte025' ,'aonte031' ,'aonte032', 'aonte033', 'aonte034' ,'aonte035', 'aonte036', 'aonte037', 'aonte038', 'aonte039', 'aonte040', 'aonte041', 'aonte042', 'aonte043', 'aonte044', 'aonte045', 'aonte046', 'aonte047', 'aonte048', 'aonte049']]
blockgroup_raw['bg_age_gt18']=col_list.sum(axis=1) # summing population count 18 years and older
blockgroup_raw['s_bg_age_gt18']=blockgroup_raw['bg_age_gt18']/blockgroup_raw['aonte001'] # percentage of population 18 years and older over whole population (creates Nan values where the population per block group is 0)
blockgroup_raw = blockgroup_raw.rename(columns={'gisjoin': 'bg_gisjoin'})
bg_age=blockgroup_raw[['bg_gisjoin', 'bg_age_gt18','s_bg_age_gt18']]

bg_age

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7492 entries, 0 to 7491
Columns: 1001 entries, gisjoin to aoslm036
dtypes: float64(76), int64(917), object(8)
memory usage: 57.2+ MB
None


,bg_gisjoin,bg_age_gt18,s_bg_age_gt18
0,G20000109526001,1415,0.769859
1,G20000109527001,845,0.795669
2,G20000109527002,726,0.823129
3,G20000109528001,841,0.819688
4,G20000109528002,1221,0.822222
...,...,...,...
7487,G29051001277001,747,0.772492
7488,G29051001277002,784,1.000000
7489,G29051001277003,318,0.736111
7490,G29051001278001,1598,0.820329


### prepare cross-walk data 

In [6]:
##b_KC_6counties
b_KC_6counties = pd.read_stata(dta_path_6_counties)
b_KC_6counties.columns = b_KC_6counties.columns.str.lower()

##JOIN
b_bg_county_crosswalk = pd.read_stata(dta_path_county_crosswalk)
b_bg_county_crosswalk.columns = b_bg_county_crosswalk.columns.str.lower()
b_bg_county_crosswalk = b_bg_county_crosswalk[['b_gisjoin', 'bg_gisjoin']] # cutting down on unnecessary columns

b_KC_6counties
b_bg_county_crosswalk


,b_gisjoin,bg_gisjoin
0,G20009100500001000,G20009100500001
1,G20009100500001001,G20009100500001
2,G20009100500001002,G20009100500001
3,G20009100500001003,G20009100500001
4,G20009100500001004,G20009100500001
...,...,...
37696,G29016509800001121,G29016509800001
37697,G29016509800001122,G29016509800001
37698,G29016509800001123,G29016509800001
37699,G29016509800001124,G29016509800001


### Merge block and block-group data

In [7]:
#merge B and BG level data, calculate target_density 
merge1=pd.merge(block_data, b_KC_6counties,on=['b_gisjoin' ])  # merge to restrict area to 6 counties in KC (426161 to 37701)
merge2=pd.merge(merge1, b_bg_county_crosswalk,on=['b_gisjoin' ])  # block block-group county crosswalk file
merge3=pd.merge(merge2, bg_age,on=['bg_gisjoin' ])  # add bg-level age
merge4=pd.merge(merge3, bg_educ_inc,on=['bg_gisjoin' ])  # add bg-level bg_educ_inc

# merge4.isna().sum().sort_values(ascending=False) # checking for Nan values
merge4 = merge4.fillna(0) # replacing Nan values with 0 
                          # (admissible, because those came into being by devision by 0 where the population count per block group was 0)
                          # (the percentage of target populaton in block groups without any population should be 0, presupposedly the data is correct)

merge4

,b_gisjoin,state,county,b_population,fid,shape_area,bg_gisjoin,bg_age_gt18,s_bg_age_gt18,bg_educ_lt_bachelars,bg_inc_lt_45000,s_bg_educ_lt_bachelars,s_bg_inc_lt_45000
0,G20009100500001000,Kansas,Johnson County,52,0,27825.763672,G20009100500001,480,0.914286,237,130,0.697059,0.546218
1,G20009100500001001,Kansas,Johnson County,47,1,26903.826172,G20009100500001,480,0.914286,237,130,0.697059,0.546218
2,G20009100500001002,Kansas,Johnson County,51,2,26385.476562,G20009100500001,480,0.914286,237,130,0.697059,0.546218
3,G20009100500001003,Kansas,Johnson County,58,3,27075.373047,G20009100500001,480,0.914286,237,130,0.697059,0.546218
4,G20009100500001004,Kansas,Johnson County,138,4,74489.484375,G20009100500001,480,0.914286,237,130,0.697059,0.546218
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37696,G29016509800001121,Missouri,Platte County,0,37696,9910.267578,G29016509800001,0,0.000000,0,0,0.000000,0.000000
37697,G29016509800001122,Missouri,Platte County,0,37697,47732.625000,G29016509800001,0,0.000000,0,0,0.000000,0.000000
37698,G29016509800001123,Missouri,Platte County,0,37698,14295.159180,G29016509800001,0,0.000000,0,0,0.000000,0.000000
37699,G29016509800001124,Missouri,Platte County,0,37699,8224.662109,G29016509800001,0,0.000000,0,0,0.000000,0.000000


### Calculate joint density of target population
In this case, we assume indepencence between the variables income, age and educational status, thus getting the joint probability P(I,E,A|bg)=P(I|bg)P(E|bg)Pr(A|bg).

In [8]:
whole_population_6_counties=merge4['b_population'].sum()
whole_area_6_counties = 2589973.632302

# joint probability age*educ*income of target population
merge4['s_joint_age_educ_inc']=merge4['s_bg_age_gt18']*merge4['s_bg_educ_lt_bachelars']*merge4['s_bg_inc_lt_45000']

# block-level estimate of target population
merge4['b_target_pop']=merge4['b_population']*merge4['s_joint_age_educ_inc']

# calculating density and distribution per sq miles
merge4['target_density']=(merge4['b_target_pop'] / whole_population_6_counties).fillna(0)
merge4['target_pop_per_sq_mile']=merge4['b_target_pop'] / merge4['shape_area'] / whole_area_6_counties

merge4


,b_gisjoin,state,county,b_population,fid,shape_area,bg_gisjoin,bg_age_gt18,s_bg_age_gt18,bg_educ_lt_bachelars,bg_inc_lt_45000,s_bg_educ_lt_bachelars,s_bg_inc_lt_45000,s_joint_age_educ_inc,b_target_pop,target_density,target_pop_per_sq_mile
0,G20009100500001000,Kansas,Johnson County,52,0,27825.763672,G20009100500001,480,0.914286,237,130,0.697059,0.546218,0.348111,18.101772,0.000009,2.511763e-10
1,G20009100500001001,Kansas,Johnson County,47,1,26903.826172,G20009100500001,480,0.914286,237,130,0.697059,0.546218,0.348111,16.361217,0.000008,2.348044e-10
2,G20009100500001002,Kansas,Johnson County,51,2,26385.476562,G20009100500001,480,0.914286,237,130,0.697059,0.546218,0.348111,17.753661,0.000009,2.597931e-10
3,G20009100500001003,Kansas,Johnson County,58,3,27075.373047,G20009100500001,480,0.914286,237,130,0.697059,0.546218,0.348111,20.190439,0.000010,2.879227e-10
4,G20009100500001004,Kansas,Johnson County,138,4,74489.484375,G20009100500001,480,0.914286,237,130,0.697059,0.546218,0.348111,48.039319,0.000024,2.490041e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37696,G29016509800001121,Missouri,Platte County,0,37696,9910.267578,G29016509800001,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
37697,G29016509800001122,Missouri,Platte County,0,37697,47732.625000,G29016509800001,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
37698,G29016509800001123,Missouri,Platte County,0,37698,14295.159180,G29016509800001,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
37699,G29016509800001124,Missouri,Platte County,0,37699,8224.662109,G29016509800001,0,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00


#### Working comment: Assuming independence of our variables (income, age and educational status) plus the very strict definition of household income instead of individual income makes us miss most of the target population.

In [9]:
merge4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37701 entries, 0 to 37700
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   b_gisjoin               37701 non-null  object 
 1   state                   37701 non-null  object 
 2   county                  37701 non-null  object 
 3   b_population            37701 non-null  int64  
 4   fid                     37701 non-null  int32  
 5   shape_area              37701 non-null  float32
 6   bg_gisjoin              37701 non-null  object 
 7   bg_age_gt18             37701 non-null  int64  
 8   s_bg_age_gt18           37701 non-null  float64
 9   bg_educ_lt_bachelars    37701 non-null  int64  
 10  bg_inc_lt_45000         37701 non-null  int64  
 11  s_bg_educ_lt_bachelars  37701 non-null  float64
 12  s_bg_inc_lt_45000       37701 non-null  float64
 13  s_joint_age_educ_inc    37701 non-null  float64
 14  b_target_pop            37701 non-null

In [10]:
final_block_data=merge4[['fid', 'county' ,'bg_gisjoin' ,'b_gisjoin' ,'b_population', 'bg_age_gt18', 'bg_educ_lt_bachelars', 'bg_inc_lt_45000','s_bg_age_gt18', 's_bg_educ_lt_bachelars', 's_bg_inc_lt_45000', 's_joint_age_educ_inc' ,'b_target_pop', 'target_pop_per_sq_mile', 'target_density']]
final_block_data

,fid,county,bg_gisjoin,b_gisjoin,b_population,bg_age_gt18,bg_educ_lt_bachelars,bg_inc_lt_45000,s_bg_age_gt18,s_bg_educ_lt_bachelars,s_bg_inc_lt_45000,s_joint_age_educ_inc,b_target_pop,target_pop_per_sq_mile,target_density
0,0,Johnson County,G20009100500001,G20009100500001000,52,480,237,130,0.914286,0.697059,0.546218,0.348111,18.101772,2.511763e-10,0.000009
1,1,Johnson County,G20009100500001,G20009100500001001,47,480,237,130,0.914286,0.697059,0.546218,0.348111,16.361217,2.348044e-10,0.000008
2,2,Johnson County,G20009100500001,G20009100500001002,51,480,237,130,0.914286,0.697059,0.546218,0.348111,17.753661,2.597931e-10,0.000009
3,3,Johnson County,G20009100500001,G20009100500001003,58,480,237,130,0.914286,0.697059,0.546218,0.348111,20.190439,2.879227e-10,0.000010
4,4,Johnson County,G20009100500001,G20009100500001004,138,480,237,130,0.914286,0.697059,0.546218,0.348111,48.039319,2.490041e-10,0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37696,37696,Platte County,G29016509800001,G29016509800001121,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
37697,37697,Platte County,G29016509800001,G29016509800001122,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
37698,37698,Platte County,G29016509800001,G29016509800001123,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
37699,37699,Platte County,G29016509800001,G29016509800001124,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000


### save final output file

In [11]:
final_block_data.to_excel("data/intermediate_data/block_level_target_population_outofpython.xlsx",index=False)
final_block_data.to_csv("data/intermediate_data/block_level_target_population_outofpython.csv",index=False)